In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import pytorch_lightning as pl
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
FEW_SHOT_DATA = pathlib.Path("../data/coco_crops_few_shot")
test = ImageFolder(FEW_SHOT_DATA/"test")
DATA_MEANS, DATA_STD = [0.48145466, 0.4578275, 0.40821073], [
    0.26862954,
    0.26130258,
    0.27577711,
] 
test_transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(DATA_MEANS, DATA_STD),
            ]
        )
test = ImageFolder(FEW_SHOT_DATA / "test", transform=test_transform)
testdl =  DataLoader(test, batch_size=32, shuffle=False)


In [ ]:
import sys
sys.path.append("../")
sys.path.append("../src/")
from src.ft_clip import ClipClassifier
model = ClipClassifier.load_from_checkpoint("/Users/marcalph/Projects/fsl/models/Vit-B/32ml1.0epoch=33.ckpt")
trainer = pl.Trainer()
trainer.test(model, dataloaders=testdl)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np

y_pred = []
y_true = []
for inputs, labels in testdl:
        output = model(inputs)
        output = (torch.max(torch.exp(output), 1)[1]).data
        y_pred.extend(output) 
        labels = labels.data
        y_true.extend(labels)


# Build precision confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred).T
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix,axis=1)[:, np.newaxis], index = [i for i in test.classes],
                     columns = [i for i in test.classes])
plt.figure(figsize = (12,7))
sns.heatmap(df_cm, annot=True, cmap="Blues")